# Summary of last lecture

In last lecture I explained some of the tools that work under the hood of our package and allow us to keep track of all numerical errors and allow us to prove some simple mathematical results with the assistance of a computer:
- Interval Arithmetic allows us to bound the range of a function on an Interval
- Interval Newton Method allows us to rigorously solve the equation $f(x)=y$ and so allows us to rigorously find the inverse values of a function
- TaylorModels allow us to compute derivatives and integrals rigorously

# Approximating the Ulam operator

In this section I will comment on the Ulam operator, its implementation and how to run the functions in our InvariantMeasures.jl package.

Given a dynamical system $T:[0,1]\to [0,1]$ the Ulam operator on a partition $P = \{I_i\}$ is a Markov chain whose transition probabilities are
$$
M(i \to j) = P(T(x)\in I_i \mid x \in I_j).
$$

This can be rewritten in the following form, if $m$ is the Lebesgue measure
$$
M(i \to j) = \frac{m(T^{-1}I_i\cap I_j)}{m(I_j)}
$$

In [ ]:
import Pkg;
# Pkg.rm("RigorousInvariantMeasures")
# normally you can only install the package as below; I am reworking a couple of interface issues in these 
# days, I will do a release during this week
#Pkg.add("RigorousInvariantMeasures")
Pkg.add(path = "https://github.com/JuliaDynamics/RigorousInvariantMeasures.jl/")

In [ ]:
Pkg.instantiate()

In [ ]:
using RigorousInvariantMeasures

The Package has already implemented different basis types, that work out of the box:
- Ulam basis for approximation in the $L^1$ norm
- Hat Basis for approximation in the $L^{\infty}$ norm (the dynamic must be Markov and regular)

the following two basis are also partially implemented

- a spline basis for approximations in $C^1$ (the dynamic must be Markov and regular)
- a Chebyshev basis for approximations in $C^1$ (the dynamic must be Markov and regular)

Today we will be working with the Ulam approximation.

In [ ]:
B = Ulam(16)

We will start by analizing a known map, for which it is simple to compare what the computer is doing with pen and paper computations.

In the file DynamicDefinition.jl of the package the methods that a Dynamic type has to satisfy for the package to work out of the box are stated.

At the moment, a generic Piecewise Dynamic type is defined in PwDynamicDefinition.jl, that should work in most cases.

In [ ]:
@doc PwMap

For our first example we will use an helper constructor, the 
```
mod1_dynamic(f)
```
where $f$ is a function from $[0,1]\to \mathbb{R}$, that constructs a PwMap without us worrying too much about the underlying structure. 

In [ ]:
D = mod1_dynamic(x->2*x)

In [ ]:
D.branches

In [ ]:
D.branches[1].X

In [ ]:
D.branches[1].Y

In [ ]:
D.branches[1].f(0.1)

In [ ]:
derivative(D.branches[1].f, 0.1)

In [ ]:
import Pkg; Pkg.add("LaTeXStrings")
using Plots, LaTeXStrings

In [ ]:
strong_norm(B)

In [ ]:
aux_norm(B)

In [ ]:
A, BB = dfly(strong_norm(B), aux_norm(B), D)
plot(D, title="Dynamic (dfly coeffs $(round(A, sigdigits=2)), $(round(BB, sigdigits=2)))", label=L"T(x)", legend=:bottomright)

What we are going to do is to compute preimages of an interval.

In [ ]:
@doc RigorousInvariantMeasures.preimages

The function preim is doing nothing else than computing the preimages of the endpoints of $B$, using the Interval Newton method. To show this, I will compare them side to side, when computing the preimages through the first branch.

In [ ]:
B.p

In [ ]:
v, v_labels = RigorousInvariantMeasures.preimages(B.p, D; ϵ = 0.00001, max_iter = 100);
v

Once we have the preimages of the interval $I_i$, we can compute the intersection with the interval $I_j$.
As an example, we will compute the relative measure of $T^{-1}(I_1)$ in $I_1$

In [ ]:
v_labels[1], v_labels[2]

In [ ]:
preimage_of_interval_through_first_branch = (v[1], v[2])

In [ ]:
interval_I_1 = (Interval(B.p[1]), Interval(B.p[2]))

In [ ]:
RigorousInvariantMeasures.relative_measure(
    (v[1],v[2]), 
    (Interval(B.p[1]), Interval(B.p[2]))
    )

As you can see, this relative measure returns an Interval, because the preimages are wide intervals. This interval contains that the true value of $M(i\to j)$. This adds an error in our computation, but this is taken care by the package.

Please remark that you do not need to do this for each entry of the matrix, the package already exports a function that does everything for you.

In [ ]:
B = Ulam(256)
Q = DiscretizedOperator(B, D)

The Discretized operator is our Ulam Matrix (and some other objects, for other discretizations).

In [ ]:
Q.L[1, 1]

In contrast with the usual notation for Markov chains, we want densities to be multiplied on the right, so, if you are used to the usual Markov chain notations, you have to switch the row and column indexes.

We will now compute an a posteriori bound on the mixing time of the Markov chain, by iterating it on vectors of the form $(1, 0, \ldots, 0,  -1, 0, \ldots )$

In [ ]:
RigorousInvariantMeasures.opnormbound(L1, Q.L)

In [ ]:
@time norms = powernormbounds(B, D; Q=Q)

In [ ]:
plot(1:16,norms[1:16])

As you can see, our algorithm computes a rigorous bound on the mixing time of the Markov chain Q; it is possible to observe the cutoff phenomena.

Let's see what happens if we take a finer partition.

In [ ]:
B = Ulam(1024)
Q = DiscretizedOperator(B, D)
@time norms = powernormbounds(B, D; Q=Q)

In [ ]:
plot!(1:16,norms[1:16], color = :red)

As you can see, the mixing time of the Markov chain has grown as the size of the partition has grown. The spectral stability result of Liverani Keller guarantees that the mixing time grow as $\log(k)$, where $k$ is the size of the partition. 

In this case, a simple argument to show this is that if we take a partition of size $k = 2^N$ it takes $N$ iterates of the transfer operator for an element of the Ulam basis to cover all of $[0,1]$.

In [ ]:
w = invariant_vector(B, Q)

In [ ]:
error = distance_from_invariant(B, D, Q, w, norms)

## An intuition of the coarse fine method

Now, I will try to give an intuitive idea behind how the coarse-fine method works.

In [ ]:
v = zeros(1024);
v[1] = 1;

In [ ]:
v = Q*v
plot(B, v)

In [ ]:
v = Q*v
plot(B, v)

In [ ]:
for i in 1:5
    v = Q*v
end
plot(B, v)

Under iterations of the system, the support of the vector $v$ becomes bigger, so it becomes visible to coarser discretizations.

The coarse operator and the fine operator satisfy the same Lasota-Yorke inequality and this guarantees that under iterations the variation of an observable is decreasing (up to some limit).

## A nonlinear example

We will now study the map from [O. Lanford, Informal remarks on the orbit structure of discrete approximations](https://projecteuclid.org/journals/experimental-mathematics/volume-7/issue-4/Informal-remarks-on-the-orbit-structure-of-discrete-approximations-to/em/1047674149.full) which is a beautiful article on computing orbits and the path method to compute Birkhoff averages.

In [ ]:
D = mod1_dynamic(x->2*x+0.5*x*(1-x), full_branch = true)

In [ ]:
A, BB = dfly(strong_norm(B), aux_norm(B), D)
plot(D, title="Dynamic (dfly coeffs $(round(A, sigdigits=2)), $(round(BB, sigdigits=2)))", label=L"T(x)", legend=:bottomright)

In [ ]:
B = Ulam(1024)
Q = DiscretizedOperator(B, D)

In [ ]:
norms = powernormbounds(B, D; Q=Q)

In [ ]:
plot(norms)

In [ ]:
w = invariant_vector(B, Q)

In [ ]:
error = distance_from_invariant(B, D, Q, w, norms)

In [ ]:
plot(B, w)
plot!(B, error)

We use now the coarse-fine strategy.

In [ ]:
B_fine = Ulam(2^16)
Q_fine = DiscretizedOperator(B_fine, D)

In [ ]:
normQ_fine = opnormbound(B_fine, weak_norm(B_fine), Q_fine)
norms_fine = finepowernormbounds(B, B_fine, D, norms; normQ_fine=normQ_fine)
w_fine = invariant_vector(B_fine, Q_fine)
error_fine = distance_from_invariant(B_fine, D, Q_fine, w_fine, norms_fine)

In [ ]:
plot(norms)
plot!(norms_fine)

In [ ]:
plot(B_fine, w_fine)
plot!(B_fine, error_fine)

We can now compute rigorously the Lyapunov Exponent.

In [ ]:
logder = discretizationlogder(B_fine, D)

In [ ]:
integrateobservable(B_fine, logder, w_fine, error_fine)

I will show now the interface to the small-matrix method to estimate the speed of convergence in $BV$ of the original operator.

In [ ]:
weak_bound, strong_bound = convergencerateabstract(B_fine, D, norms_fine)

In [ ]:
plot(strong_bound)

## Nonrigorous experiments: spectral stability

I would like to present now some numerical experiments on the spectral stability of the Ulam operator.
To do so, we will take different partition size and do a pseudospectral plot, plotting a numerical approximation of the spectrum and of the level lines of the resolvent.

In [ ]:
import Pkg; Pkg.add("Pseudospectra")

In [ ]:
using Pseudospectra, IntervalArithmetic

In [ ]:
using IntervalArithmetic

In [ ]:
B = Ulam(16)
Q = DiscretizedOperator(B, D)
spectralportrait(Matrix(mid.(Q.L))) # the method is not defined for our DiscretizedOperators yet!!!
# so we take the matrix of the midpoints of Q

In [ ]:
B = Ulam(256)
Q = DiscretizedOperator(B, D)
spectralportrait(Matrix(mid.(Q.L)))

In [ ]:
B = Ulam(1024)
Q = DiscretizedOperator(B, D)
spectralportrait(Matrix(mid.(Q.L)))

From these experiments is possible to see that while the spectrum of the operators is different, its structure is preserved.

## A Non-Markov example

Finally, I will show the same computation in an example which is non Markov.

In [ ]:
D = PwMap([x->17*x/5, 
	x->(34*((17*x-5)/17)/25+3)*((17*x-5)/17), 
	x->(34*((17*x-10)/17)/25+3)*((17*x-10)/17), 
	x->17*((17*x-15)/17)/5], 
	[Interval(0), Interval(5)/17, Interval(10)/17, Interval(15)/17, Interval(1)],
	[Interval(0) Interval(1);
	 Interval(0) Interval(1);
	 Interval(0) Interval(1);
	 Interval(0) @interval(0.4)]
	)
A, BB = dfly(strong_norm(B), aux_norm(B), D)
plot(D, title="Dynamic (dfly coeffs $(round(A, sigdigits=2)), $(round(BB, sigdigits=2)))", label=L"T(x)", legend=:bottomright)

In [ ]:
B = Ulam(1024)
Q = DiscretizedOperator(B, D)

In [ ]:
norms = powernormbounds(B, D; Q=Q)

In [ ]:
B_fine = Ulam(2^16)
Q_fine = DiscretizedOperator(B_fine, D)

In [ ]:
normQ_fine = opnormbound(B_fine, weak_norm(B_fine), Q_fine)
norms_fine = finepowernormbounds(B, B_fine, D, norms; normQ_fine=normQ_fine)
w_fine = invariant_vector(B_fine, Q_fine)
error_fine = distance_from_invariant(B_fine, D, Q_fine, w_fine, norms_fine)

In [ ]:
plot(B_fine, w_fine)
plot!(B_fine, error_fine)

In [ ]:
logder = discretizationlogder(B_fine, D)
integrateobservable(B_fine, logder, w_fine, error_fine)

# Summary of this lecture

In this lecture I showed how to use our package to rigorously approximate the Ulam operator on a partition, how to use our package to compute the norms of this discretized operator on the space of average $0$ functions and how to use the coarse-fine approach to get estimates on the mixing time of finer operators.

This allows us to rigorously approximate the invariant density of the system, and through rigorous integration to compute the Lyapunov exponent and the Birkhoff averages of an observable with a rigorous error bound.